In [1]:
import csv, json, re
from math import radians, cos, sin, asin, sqrt
import utm
import geo
from geo import sphere
import itertools

# Create city object

In [2]:
cities = {}

with open('data/stadiums_final.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        city = row["city"]
        league = row["league"]
        team = row["team"]
        venue = row["venue"]
        
        regex = r'Point\(([-\.\d+]+)\s([-\.\d+]+)'
        matches = re.compile(regex).match(row["coords"])

        lat = matches.group(2)
        lon = matches.group(1)
        
        if city not in cities:
            cities[city] = {}
            
        if venue not in cities[city]:
            cities[city][venue] = {"coords": "", "teams":[]}
            
        cities[city][venue]["coords"] = (float(lat), float(lon))
        cities[city][venue]["teams"].append({"name": team, "league": league})

# Print city object for checking

In [3]:
## Uncomment to add the Dome in for calculations

# dome = {'coords': (38.632828, -90.188501), 'teams':[{'name': 'None', 'league': 'None'}]}

# cities['STL']['Edward Jones Dome'] = dome

print(json.dumps(cities['St. Louis'], indent=1))

print("\n\n",json.dumps(cities, indent=1))

{
 "Busch Stadium": {
  "coords": [
   38.6225,
   -90.193056
  ],
  "teams": [
   {
    "name": "St. Louis Cardinals",
    "league": "MLB"
   }
  ]
 },
 "MLS Stadium (planned)": {
  "coords": [
   38.631111,
   -90.210907
  ],
  "teams": [
   {
    "name": "MLS Team",
    "league": "MLS"
   }
  ]
 },
 "Enterprise Center": {
  "coords": [
   38.626667,
   -90.2025
  ],
  "teams": [
   {
    "name": "St. Louis Blues",
    "league": "NHL"
   }
  ]
 }
}


 {
 "Atlanta": {
  "SunTrust Park": {
   "coords": [
    33.8912,
    -84.4683
   ],
   "teams": [
    {
     "name": "Atlanta Braves",
     "league": "MLB"
    }
   ]
  },
  "Mercedes-Benz Stadium": {
   "coords": [
    33.7553544,
    -84.4010356
   ],
   "teams": [
    {
     "name": "Atlanta United FC",
     "league": "MLS"
    },
    {
     "name": "Atlanta Falcons",
     "league": "NFL"
    }
   ]
  },
  "State Farm Arena": {
   "coords": [
    33.757222222,
    -84.396388888
   ],
   "teams": [
    {
     "name": "Atlanta Hawks",


## detailed_combos_of_3(city, venues)

This function takes the city object and returns a list of objects with all possible combinations of three venues.

In [4]:
def detailed_combos_of_3(city, venues):
    city_combos = []
    combos = itertools.combinations(venues, 3)
    
    for combo in combos:

        curr_list = {}
        new_combo = {"city": city, "stadiums":[]}
        for item in combo:
            new_combo["stadiums"].append({"name":item,"coords":venues[item]["coords"],"teams":venues[item]["teams"]})
        
        city_combos.append(new_combo)
        
    return city_combos


# Calculate

Here we're calculating and displaying all possible combinations of three stadiums in a city.

In [5]:
all_detailed_combos_of_3 = []

for city, venues in cities.items():        
    combos = detailed_combos_of_3(city, venues)
    for item in combos:
        all_detailed_combos_of_3.append(item)
        


for item in all_detailed_combos_of_3:
    print(item["city"])
    for stadium in item["stadiums"]:
        print('\t',stadium["name"])
    print('\n')

Atlanta
	 SunTrust Park
	 Mercedes-Benz Stadium
	 State Farm Arena


Boston
	 Fenway Park
	 Gillette Stadium
	 TD Garden


Chicago
	 Wrigley Field
	 Guaranteed Rate Field
	 Soldier Field


Chicago
	 Wrigley Field
	 Guaranteed Rate Field
	 United Center


Chicago
	 Wrigley Field
	 Soldier Field
	 United Center


Chicago
	 Guaranteed Rate Field
	 Soldier Field
	 United Center


Cincinnati
	 Great American Ball Park
	 West End Stadium (planned)
	 Paul Brown Stadium


Cleveland
	 Progressive Field
	 Rocket Mortgage FieldHouse
	 FirstEnergy Stadium


Dallas
	 Globe Life Field in Arlington (planned)
	 Toyota Stadium
	 American Airlines Center


Dallas
	 Globe Life Field in Arlington (planned)
	 Toyota Stadium
	 AT&T Stadium


Dallas
	 Globe Life Field in Arlington (planned)
	 American Airlines Center
	 AT&T Stadium


Dallas
	 Toyota Stadium
	 American Airlines Center
	 AT&T Stadium


Denver
	 Coors Field
	 Dick's Sporting Goods Park
	 Pepsi Center


Denver
	 Coors Field
	 Dick's Sporting Goo

# Convert lat/lng to x/y via utm

Next we need to convert lat/lng coordinates to UTM coordinates. UTM lets us use a Cartesian system to calculate a circle.

In [6]:
# import utm
# def Average(lst): 
#     return sum(lst) / len(lst) 

# def get_average_latitude(item):
#     lats = []
#     for stadium in item["stadiums"]:
#         lats.append(stadium["coords"][0])


#     return Average(lats)

for item in all_detailed_combos_of_3:
    
    for stadium in item['stadiums']:
        
        stad_utm = utm.from_latlon(stadium["coords"][0],stadium["coords"][1])
        stadium["utm"] = stad_utm
        
for item in all_detailed_combos_of_3:
    print(item["city"])
    for stadium in item["stadiums"]:
        print('\t',stadium["name"], ' — ', stadium['coords'], stadium['utm'])
    print('\n')

Atlanta
	 SunTrust Park  —  (33.8912, -84.4683) (734123.6968077851, 3752977.9901943556, 16, 'S')
	 Mercedes-Benz Stadium  —  (33.7553544, -84.4010356) (740726.4151937661, 3738065.6396332546, 16, 'S')
	 State Farm Arena  —  (33.757222222, -84.396388888) (741151.6913767095, 3738283.6822849135, 16, 'S')


Boston
	 Fenway Park  —  (42.34625, -71.09775) (327210.6966711702, 4690352.349737232, 19, 'T')
	 Gillette Stadium  —  (42.090944444, -71.264344444) (312734.6057034444, 4662354.851264232, 19, 'T')
	 TD Garden  —  (42.366302777, -71.062227777) (330190.76254729007, 4692507.476135686, 19, 'T')


Chicago
	 Wrigley Field  —  (41.948038, -87.65568) (445653.8583988176, 4644214.8395175515, 16, 'T')
	 Guaranteed Rate Field  —  (41.83, -87.633888888) (447363.2129664589, 4631095.793700214, 16, 'T')
	 Soldier Field  —  (41.8623, -87.616666666) (448819.0665503739, 4634671.543476239, 16, 'T')


Chicago
	 Wrigley Field  —  (41.948038, -87.65568) (445653.8583988176, 4644214.8395175515, 16, 'T')
	 Guarant

	 Honda Center  —  (33.807777777, -117.876666666) (418858.61769739806, 3741188.5046726284, 11, 'S')


Los Angeles
	 Dodger Stadium  —  (34.073611111, -118.24) (385584.2057764688, 3771011.599190654, 11, 'S')
	 SoFi Stadium (planned)  —  (33.95345, -118.3392) (376256.09039340564, 3757802.422332018, 11, 'S')
	 Honda Center  —  (33.807777777, -117.876666666) (418858.61769739806, 3741188.5046726284, 11, 'S')


Los Angeles
	 Banc of California Stadium  —  (34.013, -118.285) (381347.3627320469, 3764341.7468577353, 11, 'S')
	 Dignity Health Sports Park  —  (33.864444444, -118.261111111) (383350.85711694846, 3747841.4284279128, 11, 'S')
	 Staples Center  —  (34.043055555, -118.267222222) (383030.2617748411, 3767654.113361171, 11, 'S')


Los Angeles
	 Banc of California Stadium  —  (34.013, -118.285) (381347.3627320469, 3764341.7468577353, 11, 'S')
	 Dignity Health Sports Park  —  (33.864444444, -118.261111111) (383350.85711694846, 3747841.4284279128, 11, 'S')
	 SoFi Stadium (planned)  —  (33.95

	 Oakland Ballpark (planned)  —  (37.79575, -122.283333) (563095.9754627141, 4183395.294750121, 10, 'S')
	 Chase Center  —  (37.768056, -122.3875) (553945.0530943322, 4180257.4621464484, 10, 'S')
	 Levi's Stadium  —  (37.4034, -121.97) (591160.2297219804, 4140122.1165989577, 10, 'S')


San Francisco
	 Oakland Ballpark (planned)  —  (37.79575, -122.283333) (563095.9754627141, 4183395.294750121, 10, 'S')
	 Chase Center  —  (37.768056, -122.3875) (553945.0530943322, 4180257.4621464484, 10, 'S')
	 SAP Center  —  (37.332777777, -121.901111111) (597348.6551370284, 4132355.7061418714, 10, 'S')


San Francisco
	 Oakland Ballpark (planned)  —  (37.79575, -122.283333) (563095.9754627141, 4183395.294750121, 10, 'S')
	 Levi's Stadium  —  (37.4034, -121.97) (591160.2297219804, 4140122.1165989577, 10, 'S')
	 SAP Center  —  (37.332777777, -121.901111111) (597348.6551370284, 4132355.7061418714, 10, 'S')


San Francisco
	 Oracle Park  —  (37.778611, -122.389167) (553790.5853240027, 4181427.572894698, 1

## Check to make sure stadiums are in the same UTM zone

If stadiums aren't in the same UTM zone, coordinates will be off.

In [7]:
for item in all_detailed_combos_of_3:
    utf_zones = []
    for stadium in item["stadiums"]:
        utf_zones.append((stadium['utm'][2],stadium['utm'][3]))
        
    matches = []
    matches.append(utf_zones[0] == utf_zones[1])
    matches.append(utf_zones[1] == utf_zones[2])
    matches.append(utf_zones[0] == utf_zones[2])
    
    if False in matches:
        print('{}:'.format(item['city']))
        print('\t{}'.format(item["stadiums"]))

# Smallest enclosing circle

Here I'll attempt to calculate the radius of the smallest enclosing circle.

In [8]:
# 
# Smallest enclosing circle - Library (Python)
# 
# Copyright (c) 2018 Project Nayuki
# https://www.nayuki.io/page/smallest-enclosing-circle
# 
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# 
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
# 
# You should have received a copy of the GNU Lesser General Public License
# along with this program (see COPYING.txt and COPYING.LESSER.txt).
# If not, see <http://www.gnu.org/licenses/>.
# 

import math, random


# Data conventions: A point is a pair of floats (x, y). A circle is a triple of floats (center x, center y, radius).

# Returns the smallest circle that encloses all the given points. Runs in expected O(n) time, randomized.
# Input: A sequence of pairs of floats or ints, e.g. [(0,5), (3.1,-2.7)].
# Output: A triple of floats representing a circle.
# Note: If 0 points are given, None is returned. If 1 point is given, a circle of radius 0 is returned.
# 
# Initially: No boundary points known
def make_circle(points):
	# Convert to float and randomize order
	shuffled = [(float(x), float(y)) for (x, y) in points]
	random.shuffle(shuffled)
	
	# Progressively add points to circle or recompute circle
	c = None
	for (i, p) in enumerate(shuffled):
		if c is None or not is_in_circle(c, p):
			c = _make_circle_one_point(shuffled[ : i + 1], p)
	return c


# One boundary point known
def _make_circle_one_point(points, p):
	c = (p[0], p[1], 0.0)
	for (i, q) in enumerate(points):
		if not is_in_circle(c, q):
			if c[2] == 0.0:
				c = make_diameter(p, q)
			else:
				c = _make_circle_two_points(points[ : i + 1], p, q)
	return c


# Two boundary points known
def _make_circle_two_points(points, p, q):
	circ = make_diameter(p, q)
	left  = None
	right = None
	px, py = p
	qx, qy = q
	
	# For each point not in the two-point circle
	for r in points:
		if is_in_circle(circ, r):
			continue
		
		# Form a circumcircle and classify it on left or right side
		cross = _cross_product(px, py, qx, qy, r[0], r[1])
		c = make_circumcircle(p, q, r)
		if c is None:
			continue
		elif cross > 0.0 and (left is None or _cross_product(px, py, qx, qy, c[0], c[1]) > _cross_product(px, py, qx, qy, left[0], left[1])):
			left = c
		elif cross < 0.0 and (right is None or _cross_product(px, py, qx, qy, c[0], c[1]) < _cross_product(px, py, qx, qy, right[0], right[1])):
			right = c
	
	# Select which circle to return
	if left is None and right is None:
		return circ
	elif left is None:
		return right
	elif right is None:
		return left
	else:
		return left if (left[2] <= right[2]) else right


def make_diameter(a, b):
	cx = (a[0] + b[0]) / 2.0
	cy = (a[1] + b[1]) / 2.0
	r0 = math.hypot(cx - a[0], cy - a[1])
	r1 = math.hypot(cx - b[0], cy - b[1])
	return (cx, cy, max(r0, r1))


def make_circumcircle(a, b, c):
	# Mathematical algorithm from Wikipedia: Circumscribed circle
	ox = (min(a[0], b[0], c[0]) + max(a[0], b[0], c[0])) / 2.0
	oy = (min(a[1], b[1], c[1]) + max(a[1], b[1], c[1])) / 2.0
	ax = a[0] - ox;  ay = a[1] - oy
	bx = b[0] - ox;  by = b[1] - oy
	cx = c[0] - ox;  cy = c[1] - oy
	d = (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by)) * 2.0
	if d == 0.0:
		return None
	x = ox + ((ax*ax + ay*ay) * (by - cy) + (bx*bx + by*by) * (cy - ay) + (cx*cx + cy*cy) * (ay - by)) / d
	y = oy + ((ax*ax + ay*ay) * (cx - bx) + (bx*bx + by*by) * (ax - cx) + (cx*cx + cy*cy) * (bx - ax)) / d
	ra = math.hypot(x - a[0], y - a[1])
	rb = math.hypot(x - b[0], y - b[1])
	rc = math.hypot(x - c[0], y - c[1])
	return (x, y, max(ra, rb, rc))


_MULTIPLICATIVE_EPSILON = 1 + 1e-14

def is_in_circle(c, p):
	return c is not None and math.hypot(p[0] - c[0], p[1] - c[1]) <= c[2] * _MULTIPLICATIVE_EPSILON


# Returns twice the signed area of the triangle defined by (x0, y0), (x1, y1), (x2, y2).
def _cross_product(x0, y0, x1, y1, x2, y2):
	return (x1 - x0) * (y2 - y0) - (y1 - y0) * (x2 - x0)

# Make circles

Here we're taking each set of three stadiums and generating the smallest enclosing circle.

The smallest enclosing circle is how we've chosen to define "within X miles of". For a right or acute triangle, the smallest enclosing circle will have all three points on its boundary. For an obtuse triangle, the longest side of the triangle will be the circle's diameter, and the point opposite the longest side will lie inside the circle.

In [9]:
for item in all_detailed_combos_of_3:
    utm_points = []
    for stadium in item["stadiums"]:
        utm_points.append([stadium['utm'][0],stadium['utm'][1]])
        
    circle = make_circle(utm_points)
    
    utm_zone = item['stadiums'][0]['utm'][2:4]
    
    utm_center = circle[0:2] + utm_zone
    
    latlon_center = utm.to_latlon(*utm_center)
    item['center'] = latlon_center
    item['radius'] = circle[2]
    
    print(item['city'],item['center'])

Atlanta (33.823280822595585, -84.43464059251978)
Boston (42.228664633361376, -71.16350229901016)
Chicago (41.88901988997973, -87.64477410142673)
Chicago (41.88901988997973, -87.64477410142673)
Chicago (41.90517090700565, -87.6361601464948)
Chicago (41.855279655534154, -87.65401982783956)
Cincinnati (39.10444463621609, -84.51444225285002)
Cleveland (41.5009724469901, -81.69236057751657)
Dallas (32.950978743134485, -96.96001628787153)
Dallas (32.95119184193669, -96.96433097302906)
Dallas (32.76924769604703, -96.95155620417212)
Dallas (32.95119184193669, -96.96433097302906)
Denver (39.77709779993783, -104.94974602054063)
Denver (39.77473998071462, -104.95600078034691)
Denver (39.750000721633796, -105.00708447432375)
Denver (39.77473998071462, -104.95600078034691)
Detroit (42.34055007610185, -83.05022840150163)
Houston (29.75276911833123, -95.35754006236365)
Houston (29.72083569927604, -95.38320606974912)
Houston (29.72083569927604, -95.38320606974912)
Houston (29.718463812147156, -95.3816

#  Getting distance and bearing

Now that we have a center point for a circle, we can calculate the locations of all the other stadiums relative to the center point. We do this using `geo-py`, which gives us the great-circle distance and bearing.

Note that we're calculating a distance and bearing for each stadium in each set of three, relative to the center point for that set. So the same stadium will have different values depending on the other members of its set.

In [10]:
for item in all_detailed_combos_of_3:
    print(item['city'])
    for stadium in item["stadiums"]:
        
        stadium['dist'] = geo.sphere.distance((item['center'][1],item['center'][0]),(stadium['coords'][1],stadium['coords'][0]))

        stadium['bearing'] = geo.sphere.bearing((item['center'][1],item['center'][0]),(stadium['coords'][1],stadium['coords'][0]))
        
        print('\t',stadium['name'])
        print('\t\t', stadium['dist'],stadium['bearing'],'\n')

Atlanta
	 SunTrust Park
		 8166.832666287331 337.64005656322274 

	 Mercedes-Benz Stadium
		 8166.604260633992 157.64008870179146 

	 State Farm Arena
		 8151.710326157462 154.29053361324839 

Boston
	 Fenway Park
		 14149.490879828747 22.451543479054294 

	 Gillette Stadium
		 17424.193233675294 208.52609942314348 

	 TD Garden
		 17424.511642790225 28.522925955223286 

Chicago
	 Wrigley Field
		 6624.269946543914 352.17452767864364 

	 Guaranteed Rate Field
		 6624.3463463491435 172.17501801315973 

	 Soldier Field
		 3774.022849449739 141.9203870412697 

Chicago
	 Wrigley Field
		 6624.269946543914 352.17452767864364 

	 Guaranteed Rate Field
		 6624.3463463491435 172.17501801315973 

	 United Center
		 2608.9076070305205 248.86286652815534 

Chicago
	 Wrigley Field
		 5032.728247598543 341.290828042305 

	 Soldier Field
		 5032.777540857398 161.29119473210392 

	 United Center
		 4169.955503105174 228.9877614130639 

Chicago
	 Guaranteed Rate Field
		 3268.407703263262 149.31476969

		 22843.832854019525 288.8291164209088 

	 Honda Center
		 22850.11193166674 112.72327781948667 

Miami
	 Marlins Park
		 1042.3964674414444 148.0005395302419 

	 Miami Freedom Park (planned)
		 3758.222585207927 277.8437803332604 

	 American Airlines Arena
		 3758.182434205589 97.84384249526079 

Miami
	 Marlins Park
		 10053.414751437786 174.52651838985798 

	 Miami Freedom Park (planned)
		 9227.425320783677 201.0646844248286 

	 Hard Rock Stadium
		 10053.311620365866 354.52749126447 

Miami
	 Marlins Park
		 21794.400064684825 165.9515761024179 

	 Miami Freedom Park (planned)
		 19773.228965414088 177.05914789789153 

	 BB&T Center
		 21794.045309738776 345.9535747094357 

Miami
	 Marlins Park
		 10151.544770730468 182.4468153647307 

	 American Airlines Arena
		 10147.570401018673 164.35076049823897 

	 Hard Rock Stadium
		 10148.567560088853 346.61699186621155 

Miami
	 Marlins Park
		 21648.048967506158 170.15201669687792 

	 American Airlines Arena
		 22055.46579517833 161.

	 Prudential Center
		 11628.550280212368 247.46299611309198 

New York City
	 Yankee Stadium
		 11637.867440845324 330.545244020924 

	 Barclays Center
		 11616.619248023757 238.01783553376748 

	 Belmont Park Arena (planned)
		 11606.500789723646 104.41044528169073 

New York City
	 Yankee Stadium
		 6447.916743680767 69.24666627157853 

	 Madison Square Garden
		 6469.696620211263 176.68503379891558 

	 MetLife Stadium
		 6445.091684731399 274.8609247944163 

New York City
	 Yankee Stadium
		 11591.475246530701 62.72168087940537 

	 Madison Square Garden
		 5782.431031977901 126.43699064818247 

	 Prudential Center
		 11591.818014092938 242.72078616076982 

New York City
	 Yankee Stadium
		 11582.598073019166 329.901040833644 

	 Madison Square Garden
		 11549.906710685154 276.39384841606005 

	 Belmont Park Arena (planned)
		 11552.062036341627 105.0594270766457 

New York City
	 Yankee Stadium
		 11591.475246530701 62.72168087940537 

	 MetLife Stadium
		 4163.558637737232 328.941

		 30865.600500431716 318.6378970052813 

San Francisco
	 Oakland Ballpark (planned)
		 29327.236032341854 327.3959544345012 

	 Avaya Stadium
		 29327.41099469922 147.39370337640025 

	 Levi's Stadium
		 22300.622530033037 148.08003169155666 

San Francisco
	 Oakland Ballpark (planned)
		 30761.80243623958 326.8003553761381 

	 Avaya Stadium
		 27896.41137487585 148.06556293733559 

	 SAP Center
		 30761.95693074448 146.7979823223891 

San Francisco
	 Oakland Ballpark (planned)
		 25098.787766317408 338.41336149137555 

	 Chase Center
		 27374.742737491935 317.7843833961123 

	 Levi's Stadium
		 27374.669283865387 137.78288134867915 

San Francisco
	 Oakland Ballpark (planned)
		 29898.908946263226 335.75448708715254 

	 Chase Center
		 32331.15447048156 318.46344457116857 

	 SAP Center
		 32331.028156823555 138.46152889406517 

San Francisco
	 Oakland Ballpark (planned)
		 30761.80243623958 326.8003553761381 

	 Levi's Stadium
		 20874.50605747651 149.0225352432161 

	 SAP Center
		

# Trimming the list

We're only interested in the closest grouping of three stadiums in each city. Sort the whole list by circle radius, then pick the grouping for each city that appears first on the list.

In [11]:
all_detailed_combos_of_3.sort(key=lambda x: x['radius'])    

cities = []

final = []

for item in all_detailed_combos_of_3:
    if item["city"] not in cities:
        cities.append(item["city"])
        final.append(item)

for item in final:
    print(item["city"], item['center'])
    for stadium in item["stadiums"]:
        print('\t',stadium["name"], ' — ', stadium['dist'], stadium['bearing'])
    print('\n')


Philadelphia (39.90347225593902, -75.1691667617519)
	 Citizens Bank Park  —  353.6563668858975 42.06631299212847
	 Wells Fargo Center  —  353.6564961732867 222.0662930306919
	 Lincoln Financial Field  —  326.0659552309933 154.14781425978106


Detroit (42.34055007610185, -83.05022840150163)
	 Comerica Park  —  203.30469980675733 139.16830125134277
	 Little Caesars Arena  —  388.7974166299166 279.05044313794997
	 Ford Field  —  388.9046928776768 99.04730402086301


Houston (29.75276911833123, -95.35754006236365)
	 Minute Maid Park  —  502.2474241150517 22.42200637491061
	 BBVA Stadium  —  500.2255253689722 97.26664631038847
	 Toyota Center  —  500.3155404210175 244.0479878076863


Cleveland (41.5009724469901, -81.69236057751657)
	 Progressive Field  —  821.2775935436111 134.08858104000205
	 Rocket Mortgage FieldHouse  —  623.1426426554688 144.87383314590693
	 FirstEnergy Stadium  —  821.2729201596326 314.0887305989739


Minneapolis-St. Paul (44.97778395230943, -93.26821581986427)
	 Targe

## Shift stadiums to St. Louis

Finally, we're taking all the stadiums and shifting them to St. Louis, centered around St. Louis' center point.

We have the distance and bearing for each stadium, relative to the center point of its group. We can instead start at St. Louis' center point, but use the same distance and bearing to find where that stadium would be if it were centered in St. Louis.

We're building a JSON file to use in our mapping, which is what we output at the end.

In [12]:
for item in final:
    if item['city'] == 'St. Louis':
        center = (item['center'][1],item['center'][0])
        
output = {}

for item in final:
    city = item['city']
    city_geojson = []
    
    for stadium in item['stadiums']:
        
        name = stadium['name']
        adj_coords = geo.sphere.destination((center),stadium['dist'],stadium['bearing'])
        

        geojson = {"type": "Feature","properties":{},"geometry": {
            "type":"Point"}}
        
        geojson['properties']['name'] = name
        geojson['geometry']['coordinates'] = [adj_coords[0],adj_coords[1]]
        
        city_geojson.append(geojson)
    
    output[city] = {}
    output[city]['radius'] = item['radius']
    output[city]['geojson'] = {"type": "FeatureCollection"} 
    output[city]['geojson']['features'] = city_geojson
    

print(json.dumps(output,indent=1))

# with open('stadiums.json', 'w') as write_file:
#     json.dump(output,write_file,indent=1)

{
 "Philadelphia": {
  "radius": 353.6290203771776,
  "geojson": {
   "type": "FeatureCollection",
   "features": [
    {
     "type": "Feature",
     "properties": {
      "name": "Citizens Bank Park"
     },
     "geometry": {
      "type": "Point",
      "coordinates": [
       -90.19925112880384,
       38.6291668360406
      ]
     }
    },
    {
     "type": "Feature",
     "properties": {
      "name": "Wells Fargo Center"
     },
     "geometry": {
      "type": "Point",
      "coordinates": [
       -90.20470639952225,
       38.624444614040414
      ]
     }
    },
    {
     "type": "Feature",
     "properties": {
      "name": "Lincoln Financial Field"
     },
     "geometry": {
      "type": "Point",
      "coordinates": [
       -90.20034217675175,
       38.62416683509773
      ]
     }
    }
   ]
  }
 },
 "Detroit": {
  "radius": 389.8241506127745,
  "geojson": {
   "type": "FeatureCollection",
   "features": [
    {
     "type": "Feature",
     "properties": {
      "n